In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction

This analysis aims to build a spam detection model using the Naive Bayes approach using the email spam [dataset](https://www.kaggle.com/veleon/ham-and-spam-dataset). 

This notebook is run on Kaggle.

# Initialize

In [4]:
# Import libraries
import os
import email
import random
import email.policy
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [7]:
# Construct a panda data frame of spam and ham email.
"""
From Spam Filter using Word Embedding & LSTM
https://www.kaggle.com/lonnieqin/spam-filter-using-word-embedding-lstm
"""

base_directory = "/kaggle/input/ham-and-spam-dataset/hamnspam/"
spam_email_names = os.listdir(base_directory + "spam")
normal_email_names = os.listdir(base_directory + "ham")

def load_email(is_spam, filename):
    directory = base_directory + ("spam" if is_spam else "ham")
    with open(os.path.join(directory, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)
    
spam_emails = [load_email(True, filename) for filename in spam_email_names]
normal_emails = [load_email(False, filename) for filename in normal_email_names]
random.shuffle(spam_emails)
random.shuffle(normal_emails)

def process_email(emails, label, data_dictionary, default_topic=None, validation=0):
    for mail in emails:
        payload = mail.get_payload()
        if isinstance(payload, list):
            process_email(payload, label, data_dictionary, default_topic=mail["Subject"], validation=validation)
        else:
            if "Content-Type" in mail.keys():
                if "html" in mail["Content-Type"].lower():
                    try: 
                        soup = BeautifulSoup(mail.get_content())
                        topic = mail["Subject"]
                        if topic == None:
                            topic = default_topic
                        content = soup.body.text
                        data_dictionary["topic"].append(topic)
                        data_dictionary["content"].append(content)
                        data_dictionary["label"].append(label)
                        data_dictionary["validation"].append(validation)
                    except:
                        pass
                elif "plain" in mail["Content-Type"].lower():
                    try: 
                        topic = mail["Subject"]
                        if topic == None:
                            topic = default_topic
                        content = mail.get_content()
                        data_dictionary["topic"].append(topic)
                        data_dictionary["content"].append(content)
                        data_dictionary["label"].append(label)
                        data_dictionary["validation"].append(validation)
                    except:
                        pass
                else:
                    pass
                
validation_split = 0.15

data_dictionary = {"topic": [], "content": [], "label": [], "validation": []}
for i in range(5):
    validation_count = int(validation_split * len(spam_emails))
    process_email(spam_emails[: validation_count], 1, data_dictionary, validation=1)
    process_email(spam_emails[validation_count: ], 1, data_dictionary, validation=0)
validation_count = int(validation_split * len(normal_emails))
process_email(normal_emails[: validation_count], 0, data_dictionary, validation=1)
process_email(normal_emails[validation_count: ], 0, data_dictionary, validation=0)
df = pd.DataFrame(data_dictionary)
df.dropna(inplace=True)
df = df.sample(frac=1)

df.head(30)

,topic,content,label,validation
3667,Nobel prize for British and US scientists,"URL: http://www.newsisfree.com/click/-2,865571...",0,0
2672,Adams linked to IRA death squad,"URL: http://www.newsisfree.com/click/-1,838114...",0,1
4575,Re: The Curse of India's Socialism,I think that this and other articles confuse S...,0,0
447,UK's Leading PC Specialist,Nationwide PC Repairs and Upgrades\n\n*No call...,1,0
2605,Danger Hiptop reviewed,URL: http://boingboing.net/#85511986\nDate: No...,0,1
3705,Australia declares world's largest marine reserve,"URL: http://www.newsisfree.com/click/-3,870478...",0,0
2901,Re: [ILUG] Email list management howto,Justin MacCarthy wrote:\n> I think I'll ask th...,0,0
1732,Save now,"*****Write down*****\n\n\nHello ,\n\n\nIt is t...",1,0
3848,I'm back in town...,I've been working on salary related things the...,0,0
2502,Industry Forum #136,\n\n\n\n\n\n\n\n\n\n\n\n \n\n\nThe Industry ...,1,0


# References
* [Spam Filter using Word Embedding & LSTM](https://www.kaggle.com/lonnieqin/spam-filter-using-word-embedding-lstm)